## Create simple agent

In [1]:
from strands import Agent

# Create an agent with default settings
agent = Agent()

# Ask the agent a question
agent("Tell me about agentic AI")

ClientError: An error occurred (UnrecognizedClientException) when calling the ConverseStream operation: The security token included in the request is invalid.

In [2]:
agent.model.config

{'model_id': 'us.anthropic.claude-sonnet-4-20250514-v1:0',
 'include_tool_result_status': 'auto'}

## Let's use LLMs via Ollama
- install Ollama
- make sure it is running (http://localhost:11434)

In [ ]:
from strands import Agent
from strands.models.ollama import OllamaModel

# Create an Ollama model instance
ollama_model = OllamaModel(
    host="http://localhost:11434",  # Ollama server address
    model_id="llama3"# Specify which model to use
)

# Create an agent using the Ollama model
agent = Agent(model=ollama_model)

# Use the agent
agent("Tell me about Nepal's flag.")

The national flag of Nepal is a unique and distinctive symbol of the country. The flag features a crimson red background with a yellow shape in the upper portion, known as the "spatula" or "dundha". The spatula has a triangular shape with three equal parts: two outer parts are curved outward and one central part is curved inward.

The colors of the Nepalese flag have specific meanings:

* Crimson red represents the brave and courageous people of Nepal.
* Yellow represents the rich cultural heritage and the brightness of the country's future.

The spatula symbolizes the country's history, culture, and traditions. It is said to resemble a sword or a shield, which reflects the bravery and strength of the Nepalese people.

The flag was adopted in 1962, following Nepal's transition from a monarchical system to a republic. The design is inspired by the traditional Nepali clothing and architecture, reflecting the country's rich cultural heritage.

Interestingly, the Nepalese flag is also one 

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The national flag of Nepal is a unique and distinctive symbol of the country. The flag features a crimson red background with a yellow shape in the upper portion, known as the "spatula" or "dundha". The spatula has a triangular shape with three equal parts: two outer parts are curved outward and one central part is curved inward.\n\nThe colors of the Nepalese flag have specific meanings:\n\n* Crimson red represents the brave and courageous people of Nepal.\n* Yellow represents the rich cultural heritage and the brightness of the country\'s future.\n\nThe spatula symbolizes the country\'s history, culture, and traditions. It is said to resemble a sword or a shield, which reflects the bravery and strength of the Nepalese people.\n\nThe flag was adopted in 1962, following Nepal\'s transition from a monarchical system to a republic. The design is inspired by the traditional Nepali clothing and architect

### Using different models

In [5]:
# Create models for different use cases
creative_model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3",
    temperature=0.8
)

factual_model = OllamaModel(
    host="http://localhost:11434",
    model_id="gemma3:1b",
    temperature=0.2
)

# Create agents with different models
creative_agent = Agent(model=creative_model)
factual_agent = Agent(model=factual_model)

### Structured output

In [6]:
from pydantic import BaseModel, Field
from strands import Agent
from strands.models.ollama import OllamaModel

class BookAnalysis(BaseModel):
    """Analyze a book's key information."""
    title: str = Field(description="The book's title")
    author: str = Field(description="The book's author")
    genre: str = Field(description="Primary genre or category")
    summary: str = Field(description="Brief summary of the book")
    rating: int = Field(description="Rating from 1-10", ge=1, le=10)

ollama_model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3",
)

agent = Agent(model=ollama_model)

result = agent.structured_output(
    BookAnalysis,
    """
    Analyze this book: "The Hitchhiker's Guide to the Galaxy" by Douglas Adams.
    It's a science fiction comedy about Arthur Dent's adventures through space
    after Earth is destroyed. It's widely considered a classic of humorous sci-fi.
    """
)

print(f"Title: {result.title}")
print(f"Author: {result.author}")
print(f"Genre: {result.genre}")
print(f"Rating: {result.rating}")

Title: Analysis of 'The Hitchhiker's Guide to the Galaxy' by Douglas Adams
Author: Douglas Adams
Genre: Science Fiction Comedy
Rating: 5


### Using tools ([already available tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/community-tools-package/))

In [ ]:
from strands import Agent
from strands.models.ollama import OllamaModel
from strands_tools import calculator, current_time

# Create an Ollama model
ollama_model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3" #you need llm that can be used for tool use
)

# Create an agent with tools
agent = Agent(
    model=ollama_model,
    tools=[calculator, current_time]
)

# Use the agent with tools
response = agent("What's the square root of 144 plus the current time?")

ResponseError: registry.ollama.ai/library/gemma3:1b does not support tools (status code: 400)

In [2]:
# Create an Ollama model
from strands.models.ollama import OllamaModel
ollama_model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3.1" #you need llm that can be used for tool use
)

In [12]:
from strands import Agent
from strands.models.ollama import OllamaModel
from strands_tools import calculator, current_time

# Create an Ollama model
ollama_model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3.1" #you need llm that can be used for tool use
)

# Create an agent with tools
agent = Agent(
    model=ollama_model,
    tools=[calculator, current_time]
)

# Use the agent with tools
response = agent("What's the square root of 144 ?")


Tool #1: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ sqrt(144)           │                                                                            │
│  │ Result    │ 12                  │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The square root of 144 is 12.

### Custom tools along with existing ones

In [14]:
from strands import Agent, tool
from strands_tools import calculator, current_time

# Define a custom tool as a Python function using the @tool decorator
@tool
def letter_counter(word: str, letter: str) -> int:
    """
    Count occurrences of a specific letter in a word.

    Args:
        word (str): The input word to search in
        letter (str): The specific letter to count

    Returns:
        int: The number of occurrences of the letter in the word
    """
    if not isinstance(word, str) or not isinstance(letter, str):
        return 0

    if len(letter) != 1:
        raise ValueError("The 'letter' parameter must be a single character")

    return word.lower().count(letter.lower())

# Create an agent with tools from the community-driven strands-tools package
# as well as our custom letter_counter tool
agent = Agent(model= ollama_model, tools=[calculator, current_time, letter_counter])

# Ask the agent a question that uses the available tools
message = """
I have 3 requests:

1. What is the time right now?
2. Calculate 3111696 / 74088
3. Tell me how many letter R's are in the word "strawberry" 🍓
"""
agent(message)


Tool #1: current_time

Tool #2: calculator

Tool #3: letter_counter


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 3111696 / 74088     │                                                                            │
│  │ Result    │ 42                  │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

1. The current time is 2025-09-06T10:35:43.864467+00:00.
2. The result of the division is 42.
3. There are 3 letter R's in the word "strawberry".

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '1. The current time is 2025-09-06T10:35:43.864467+00:00.\n2. The result of the division is 42.\n3. There are 3 letter R\'s in the word "strawberry".'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'current_time': ToolMetrics(tool={'toolUseId': 'current_time', 'name': 'current_time', 'input': {'timezone': ''}}, call_count=1, success_count=1, error_count=0, total_time=0.002892732620239258), 'letter_counter': ToolMetrics(tool={'toolUseId': 'letter_counter', 'name': 'letter_counter', 'input': {'letter': 'R', 'word': 'strawberry'}}, call_count=1, success_count=1, error_count=0, total_time=0.0006477832794189453), 'calculator': ToolMetrics(tool={'toolUseId': 'calculator', 'name': 'calculator', 'input': {'expression': '3111696 / 74088', 'mode': 'evaluate'}}, call_count=1, success_count=1, error_count=0, total_time=0.0033228397369384766)}, cycle_durations=[2.6488020420074463], traces=[<strands.tel

### MCP with Strands
- https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/mcp-tools/#model-context-protocol-mcp-tools

In [3]:
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient

# Connect to an MCP server using stdio transport
# Note: uvx command syntax differs by platform

# For macOS/Linux:
stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx", 
        args=["awslabs.aws-documentation-mcp-server@latest"]
    )
))

# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(model=ollama_model, tools=tools)
    agent("What is AWS Lambda?")


Tool #1: search_documentation
Based on the tool call output, AWS Lambda is a service offered by Amazon Web Services (AWS) that allows users to run small pieces of code, known as functions, in response to events. These functions can be written in a variety of programming languages and can be triggered by a range of events, such as changes to data or user interactions. AWS Lambda provides a serverless computing experience, meaning that the user does not need to provision or manage servers to run their code. The service automatically scales up or down based on the workload, making it a popular choice for building scalable and flexible applications.